In [21]:
# prompt: ModuleNotFoundError: No module named 'keras.preprocessing.text'
!pip install keras --upgrade
# now you can use keras.preprocessing.text


In [22]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

<ipython-input-22-23daeded08de>:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-22-23daeded08de>:24: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 53s - 183ms/step - accuracy: 0.6404 - loss: 0.8346
144/144 - 4s - 30ms/step - accuracy: 0.6693 - loss: 0.7561
0.7560551166534424
0.669287919998169
['loss', 'compile_metrics']


In [23]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)


model.save('sentiment_analysis_model.h5')

from tensorflow import keras
model = keras.models.load_model('sentiment_analysis_model.h5')

new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"]

new_text_sequences = tokenizer.texts_to_sequences(new_text)
new_text_padded = pad_sequences(new_text_sequences, maxlen=X.shape[1])


predictions = model.predict(new_text_padded)


predicted_labels = np.argmax(predictions, axis=1)

predicted_sentiments = labelencoder.inverse_transform(predicted_labels)

predicted_sentiments


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 57s - 197ms/step - accuracy: 0.6403 - loss: 0.8319


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step


array(['Negative'], dtype=object)

In [1]:
!pip install -U scikit-learn==1.4.2 scikeras==0.12.0


In [24]:


import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the data
data = pd.read_csv('Data.csv')
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

data['text'] = data['text'].str.replace('rt', ' ', regex=False)


max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = integer_encoded

X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)



def create_model(embed_dim=128, lstm_out=196, input_length=None, max_features=2000):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=input_length))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = KerasClassifier(
    model=create_model,
    verbose=0,
    model__input_length=X.shape[1],
    model__max_features=max_fatures
)

param_grid = {
    'model__embed_dim': [64, 128],
    'model__lstm_out': [64, 128],
    'batch_size': [16, 32],
    'epochs': [1, 2]

}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    error_score='raise'
)

grid_result = grid.fit(X_train, Y_train)


print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


AttributeError: Sequential model 'sequential' has no defined outputs yet.

In [25]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re

# -------------------------------
# Step 1: Load and preprocess data
# -------------------------------

data = pd.read_csv('Data.csv')
data = data[['text', 'sentiment']]

data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
data['text'] = data['text'].str.replace('rt', ' ', regex=False)

# -------------------------------
# Step 2: Tokenization
# -------------------------------

max_features = 2000  # <<< NOTE THE CORRECT VARIABLE NAME!

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# -------------------------------
# Step 3: Encode labels (NO to_categorical)
# -------------------------------

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = integer_encoded  # SciKeras expects integer labels, not one-hot

X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# -------------------------------
# Step 4: Model-building function
# -------------------------------

def create_model(embed_dim=128, lstm_out=196, input_length=None, max_features=2000):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=input_length))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

# -------------------------------
# Step 5: Wrap model for SciKeras
# -------------------------------

model = KerasClassifier(
    model=create_model,
    verbose=0,
    model__input_length=X.shape[1],
    model__max_features=max_features
)

# -------------------------------
# Step 6: Grid search
# -------------------------------

param_grid = {
    'model__embed_dim': [64, 128],
    'model__lstm_out': [64, 128],
    'batch_size': [16, 32],
    'epochs': [1, 2]
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    n_jobs=1  # safer: avoids parallelism issues while debugging
)

# -------------------------------
# Step 7: Fit Grid Search
# -------------------------------

grid_result = grid.fit(X_train, Y_train)

# -------------------------------
# Step 8: Output best results
# -------------------------------

print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/p

ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py", line 926, in _fit
    self._check_model_compatibility(y)
  File "/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py", line 549, in _check_model